In [2]:
# =============================================================================
# OCCURRENCE AND FREQUENCY OF HEAT AND COLD WAVES IN MAPUTO
# Based on WMO/IPCC/FAO/INPE standards (Enhanced Version)
# =============================================================================

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy.stats import linregress
from sklearn.preprocessing import StandardScaler
from fancyimpute import IterativeSVD
from sklearn.metrics import mean_squared_error
import calendar
import warnings

import sklearn
print(sklearn.__version__)


1.6.1


In [4]:
# === SETTINGS ===
warnings.filterwarnings('ignore')
sns.set(style='whitegrid')

input_file = "C:/Python/Waves/maputoo.xlsx"
output_path = "C:/Python/Waves/Maputo"
os.makedirs(output_path, exist_ok=True)

# === FUNCTIONS ===
def save_plot(fig, filename, dpi=600):
    fig.tight_layout()
    fig.savefig(os.path.join(output_path, filename), dpi=dpi, bbox_inches='tight')
    plt.close(fig)

def fix_year(row):
    try:
        y = int(row['YEAR'])
        dm = row['DIA_MES']
        if not calendar.isleap(y) and dm.month == 2 and dm.day == 29:
            return dm.replace(day=28, year=y)
        return dm.replace(year=y)
    except:
        return pd.NaT

def detect_wave(df, col, min_days=3):
    sequences, current = [], []
    for i, val in enumerate(df[col]):
        if val:
            current.append(i)
        elif len(current) >= min_days:
            sequences.append(current)
            current = []
        else:
            current = []
    if len(current) >= min_days:
        sequences.append(current)
    return sequences

# === 1. READ AND PREPROCESS ===
df = pd.read_excel(input_file, engine='openpyxl')
df.columns = ['YEAR', 'DIA_MES', 'T2M_MAX', 'T2M_MIN']
df['DIA_MES'] = pd.to_datetime(df['DIA_MES'], errors='coerce')
df['DATE'] = df.apply(fix_year, axis=1)
df.dropna(subset=['DATE'], inplace=True)
df.drop_duplicates(subset='DATE', inplace=True)
df['T2M_MEAN'] = (df['T2M_MAX'] + df['T2M_MIN']) / 2

# === 2. IMPUTATION + VALIDATION ===
cols = ['T2M_MAX', 'T2M_MIN', 'T2M_MEAN']
scaler = StandardScaler()

# Imputation Validation (10% masked)
df_val = df.copy()
mask = np.random.rand(len(df_val)) < 0.1
original = df_val.loc[mask, 'T2M_MAX'].copy()
df_val.loc[mask, 'T2M_MAX'] = np.nan

df_val[cols] = IterativeSVD(rank=2).fit_transform(scaler.fit_transform(df_val[cols]))
mse = mean_squared_error(original, df_val.loc[mask, 'T2M_MAX'])
rmse = np.sqrt(mse)

print(f"📉 RMSE of imputation (T2M_MAX): {rmse:.2f} °C")

# Final imputation
df[cols] = IterativeSVD(rank=2).fit_transform(scaler.fit_transform(df[cols]))

# === 3. CLIMATOLOGY (1991–2020) ===
df['DOY'] = df['DATE'].dt.dayofyear
clim_base = df[(df['YEAR'] >= 1991) & (df['YEAR'] <= 2020)]
clim = clim_base.groupby('DOY').agg(
    Tmax_Mean=('T2M_MAX', 'mean'),
    Tmax_P90=('T2M_MAX', lambda x: np.percentile(x, 90)),
    Tmax_P10=('T2M_MAX', lambda x: np.percentile(x, 10)),
    Tmin_Mean=('T2M_MIN', 'mean'),
    Tmin_P90=('T2M_MIN', lambda x: np.percentile(x, 90)),
    Tmin_P10=('T2M_MIN', lambda x: np.percentile(x, 10)),
    Tmean_Mean=('T2M_MEAN', 'mean'),
).reset_index()
df = df.merge(clim, on='DOY', how='left')

# === 4. EXTREMES AND WAVES ===
df['Tmax_Anomaly'] = df['T2M_MAX'] - df['Tmax_Mean']
df['Tmin_Anomaly'] = df['T2M_MIN'] - df['Tmin_Mean']
df['Hot_Extreme'] = df['T2M_MAX'] > df['Tmax_P90']
df['Cold_Extreme'] = df['T2M_MIN'] < df['Tmin_P10']
df['SU'] = df['T2M_MAX'] > 25

heat_waves = detect_wave(df, 'Hot_Extreme')
cold_waves = detect_wave(df, 'Cold_Extreme')
df['WSDI'] = False
df['CSDI'] = False
for s in heat_waves: df.loc[s, 'WSDI'] = True
for s in cold_waves: df.loc[s, 'CSDI'] = True

waves_df = pd.DataFrame(
    [{"Start": df.loc[s[0], 'DATE'], "End": df.loc[s[-1], 'DATE'], "Duration": len(s), "Type": "Heat"} for s in heat_waves] +
    [{"Start": df.loc[s[0], 'DATE'], "End": df.loc[s[-1], 'DATE'], "Duration": len(s), "Type": "Cold"} for s in cold_waves]
)

# === 5. INDICES AND SEASONAL AGG ===
days_per_year = df.groupby('YEAR').size()
tx90p = df.groupby('YEAR')['Hot_Extreme'].sum() / days_per_year * 100
tn10p = df.groupby('YEAR')['Cold_Extreme'].sum() / days_per_year * 100

df['MONTH'] = df['DATE'].dt.month
df['SEASON'] = df['MONTH'].map(lambda m: 'DJF' if m in [12,1,2] else 'MAM' if m in [3,4,5] else 'JJA' if m in [6,7,8] else 'SON')

seasonal = df.groupby(['YEAR', 'SEASON']).agg(
    Heat_Days=('Hot_Extreme', 'sum'),
    Cold_Days=('Cold_Extreme', 'sum'),
    Tmax_Anom_Mean=('Tmax_Anomaly', 'mean'),
    Tmin_Anom_Mean=('Tmin_Anomaly', 'mean')
).reset_index()

group = df.groupby('YEAR')
etccdi = group.agg(
    TXx=('T2M_MAX', 'max'), TXn=('T2M_MAX', 'min'),
    TNx=('T2M_MIN', 'max'), TNn=('T2M_MIN', 'min'),
    FD=('T2M_MIN', lambda x: (x < 0).sum()),
    TR=('T2M_MIN', lambda x: (x > 20).sum()),
    ID=('T2M_MAX', lambda x: (x < 0).sum())
).reset_index()
etccdi['DTR'] = group['T2M_MAX'].mean().values - group['T2M_MIN'].mean().values

# === 6. PLOTS ===

# Climatology
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(clim['DOY'], clim['Tmax_Mean'], label='Tmax Mean (1991–2020)', color='firebrick')
ax.plot(clim['DOY'], clim['Tmin_Mean'], label='Tmin Mean (1991–2020)', color='navy')
ax.fill_between(clim['DOY'], clim['Tmax_P10'], clim['Tmax_P90'], color='salmon', alpha=0.3)
ax.fill_between(clim['DOY'], clim['Tmin_P10'], clim['Tmin_P90'], color='lightblue', alpha=0.3)
ax.set_title("Daily Climatology of Tmax and Tmin")
ax.set_xlabel("Day of Year"); ax.set_ylabel("Temperature (°C)")
ax.legend(); ax.grid(True)
save_plot(fig, "Climatology_Tmax_Tmin_EN.png")

# TX90p & TN10p
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(tx90p.index, tx90p.values, marker='o', label='TX90p (%)', color='red')
ax.plot(tn10p.index, tn10p.values, marker='s', label='TN10p (%)', color='blue')
ax.set_title("Annual Frequency of Extreme Temperature Days")
ax.set_xlabel("Year"); ax.set_ylabel("Percentage of Days")
ax.legend(); ax.grid(True)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
save_plot(fig, "Extreme_Days_TX90p_TN10p_EN.png")

# Trend line for TX90p
slope, intercept, *_ = linregress(tx90p.index, tx90p.values)
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(tx90p.index, tx90p.values, 'o-', label='TX90p (%)', color='darkred')
ax.plot(tx90p.index, intercept + slope * tx90p.index, 'r--', label=f'Trend: {slope:.2f} %/yr')
ax.set_title("TX90p Trend Over Time")
ax.set_xlabel("Year"); ax.set_ylabel("TX90p (%)")
ax.legend(); ax.grid(True)
save_plot(fig, "TX90p_Trend_EN.png")

# Wave frequency
waves_df['Year'] = waves_df['Start'].dt.year
wave_freq = waves_df.groupby(['Year', 'Type']).size().unstack(fill_value=0)
fig, ax = plt.subplots(figsize=(10, 5))
wave_freq.plot(kind='bar', stacked=True, ax=ax, color=['darkorange', 'steelblue'])
ax.set_title("Annual Number of Heat and Cold Waves")
ax.set_xlabel("Year"); ax.set_ylabel("Number of Events")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
save_plot(fig, "Annual_Heat_Cold_Waves_EN.png")

# Boxplots
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(data=df, x='SEASON', y='Tmax_Anomaly', ax=ax, palette='Reds')
ax.set_title("Tmax Anomalies by Season")
save_plot(fig, "Boxplot_Tmax_Anomalies_EN.png")

fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(data=df, x='SEASON', y='Tmin_Anomaly', ax=ax, palette='Blues')
ax.set_title("Tmin Anomalies by Season")
save_plot(fig, "Boxplot_Tmin_Anomalies_EN.png")

# Decadal waves
waves_df['Decade'] = (waves_df['Year'] // 10) * 10
decade_freq = waves_df.groupby(['Decade', 'Type']).size().unstack(fill_value=0)
fig, ax = plt.subplots(figsize=(10, 6))
decade_freq.plot(kind='bar', ax=ax, color=['orangered', 'skyblue'])
ax.set_title("Heat and Cold Waves per Decade")
ax.set_ylabel("Number of Events")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
save_plot(fig, "Decadal_Waves_EN.png")

# === 7. EXPORT RESULTS ===
df.to_excel(os.path.join(output_path, "Cleaned_Full_Series.xlsx"), index=False)
clim.to_excel(os.path.join(output_path, "Climatology_1991_2020.xlsx"), index=False)
tx90p.to_frame("TX90p (%)").join(tn10p.to_frame("TN10p (%)")).to_excel(
    os.path.join(output_path, "Extreme_Indices_TX90p_TN10p.xlsx"))
seasonal.to_excel(os.path.join(output_path, "Seasonal_Summary.xlsx"), index=False)
etccdi.to_excel(os.path.join(output_path, "ETCCDI_Additional_Indices.xlsx"), index=False)
waves_df.to_excel(os.path.join(output_path, "Heat_Cold_Waves_Maputo.xlsx"), index=False)

print("\n✅ All analyses and plots completed successfully!")
print(f"📁 Output folder: {output_path}")


[IterativeSVD] Iter 1: observed MAE=0.248310
[IterativeSVD] Iter 2: observed MAE=0.013091
[IterativeSVD] Iter 3: observed MAE=0.011532
[IterativeSVD] Iter 4: observed MAE=0.009898
[IterativeSVD] Iter 5: observed MAE=0.008332
[IterativeSVD] Iter 6: observed MAE=0.006928
[IterativeSVD] Iter 7: observed MAE=0.005716
[IterativeSVD] Iter 8: observed MAE=0.004694
[IterativeSVD] Iter 9: observed MAE=0.003843
[IterativeSVD] Iter 10: observed MAE=0.003140
[IterativeSVD] Iter 11: observed MAE=0.002562
[IterativeSVD] Iter 12: observed MAE=0.002089
[IterativeSVD] Iter 13: observed MAE=0.001702
[IterativeSVD] Iter 14: observed MAE=0.001386
[IterativeSVD] Iter 15: observed MAE=0.001129
[IterativeSVD] Iter 16: observed MAE=0.000919
[IterativeSVD] Iter 17: observed MAE=0.000748
[IterativeSVD] Iter 18: observed MAE=0.000609
[IterativeSVD] Iter 19: observed MAE=0.000496
📉 RMSE of imputation (T2M_MAX): 29.12 °C


c:\Users\Dell\anaconda3\Lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[IterativeSVD] Iter 1: observed MAE=0.239434
[IterativeSVD] Iter 2: observed MAE=0.000000
[IterativeSVD] Iter 3: observed MAE=0.000000
[IterativeSVD] Iter 4: observed MAE=0.000000
[IterativeSVD] Iter 5: observed MAE=0.000000
[IterativeSVD] Iter 6: observed MAE=0.000000
[IterativeSVD] Iter 7: observed MAE=0.000000
[IterativeSVD] Iter 8: observed MAE=0.000000
[IterativeSVD] Iter 9: observed MAE=0.000000
[IterativeSVD] Iter 10: observed MAE=0.000000
[IterativeSVD] Iter 11: observed MAE=0.000000
[IterativeSVD] Iter 12: observed MAE=0.000000
[IterativeSVD] Iter 13: observed MAE=0.000000
[IterativeSVD] Iter 14: observed MAE=0.000000
[IterativeSVD] Iter 15: observed MAE=0.000000
[IterativeSVD] Iter 16: observed MAE=0.000000
[IterativeSVD] Iter 17: observed MAE=0.000000
[IterativeSVD] Iter 18: observed MAE=0.000000
[IterativeSVD] Iter 19: observed MAE=0.000000
[IterativeSVD] Iter 20: observed MAE=0.000000
[IterativeSVD] Iter 21: observed MAE=0.000000
[IterativeSVD] Iter 22: observed MAE=0.0000

C:\Users\Dell\AppData\Local\Temp\ipykernel_14700\1025612419.py:145: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
C:\Users\Dell\AppData\Local\Temp\ipykernel_14700\1025612419.py:170: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df, x='SEASON', y='Tmax_Anomaly', ax=ax, palette='Reds')
C:\Users\Dell\AppData\Local\Temp\ipykernel_14700\1025612419.py:175: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df, x='SEASON', y='Tmin_Anomaly', ax=ax, palette='Blues')



✅ All analyses and plots completed successfully!
📁 Output folder: C:/Python/Waves/Maputo
